Доступные ресурсы -Russian DALL-E :)

By russian company [Sber](https://github.com/sberbank-ai/ru-dalle).

With added translation step. You should be able to input any language.

![Russian Roulette](https://pbs.twimg.com/media/FDNZJeUXsAkmBgg?format=jpg&name=360x360)

*Example: Russian Roulette*

In [ ]:

# Text Prompt. Can be any language.
text_input = 'russian roulette'  #@param {type: "string"}´

output_path = "/content"

In [ ]:
#@title Доступные ресурсы
import multiprocessing
import torch
from psutil import virtual_memory

ram_gb = round(virtual_memory().total / 1024**3, 1)

print('CPU:', multiprocessing.cpu_count())
print('RAM GB:', ram_gb)
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device.type)

!nvidia-smi

In [ ]:
!pip install translators --upgrade
import translators as ts

def to_russian(text):
  return ts.google(text, to_language="ru"))

In [ ]:
!pip install rudalle==0.0.1rc4 > /dev/null

In [ ]:
from rudalle.pipelines import generate_images, show, super_resolution, cherry_pick_by_clip
from rudalle import get_rudalle_model, get_tokenizer, get_vae, get_realesrgan, get_ruclip
from rudalle.utils import seed_everything

In [ ]:
device = 'cuda'
dalle = get_rudalle_model('Malevich', pretrained=True, fp16=True, device=device)

In [ ]:
realesrgan = get_realesrgan('x4', device=device)
tokenizer = get_tokenizer()
vae = get_vae().to(device)
ruclip, ruclip_processor = get_ruclip('ruclip-vit-base-patch32-v5')
ruclip = ruclip.to(device)

## generation by ruDALLE

In [ ]:
text = to_russian(text_input)
pil_images = []
scores = []

seed_everything(42)

for top_k, top_p, images_num in [
    (1024, 0.98, 1),
]:
    _pil_images, _scores = generate_images(text, tokenizer, dalle, vae, top_k=top_k, images_num=images_num, top_p=top_p)
    pil_images += _pil_images
    scores += _scores

### auto-cherry-pick by ruCLIP

## super resolution

In [ ]:
sr_images = super_resolution(pil_images, realesrgan)
for image in sr_images:
  image.save(f"{output_path}/output.png")